In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

train = pd.read_csv('train_contest.csv')
test = pd.read_csv('test_contest.csv')

X = train.drop(columns=['target'])
y = train.target

index = test.index
test = test.drop(columns=['index'])

# Создаём KFold для OOF-предсказаний
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [53]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

# Пример: укажите имена категориальных колонок
categorical_cols = X.select_dtypes(include=['category', 'object']).columns.tolist()
numeric_cols = [col for col in X.columns if col not in categorical_cols]

# Препроцессинг:
# - Категориальные -> OneHotEncoder
# - Числовые -> StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_cols)
    ])

In [54]:
for feat in categorical_cols:
    X[feat] = X[feat].astype('category')
    test[feat] = test[feat].astype('category')

In [ ]:
# Список базовых моделей (подберите параметры под свои данные!)
models = [
    # Линейные модели
    ('Ridge', Pipeline([('preprocessor', preprocessor), ('model', Ridge())])),
    ('Lasso', Pipeline([('preprocessor', preprocessor), ('model', Lasso())])),
    ('ElasticNet', Pipeline([('preprocessor', preprocessor), ('model', ElasticNet())])),
    
    # Деревянные ансамбли
    ('RandomForest', Pipeline([('preprocessor', preprocessor), ('model', RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42))])),
    ('ExtraTrees', Pipeline([('preprocessor', preprocessor), ('model', ExtraTreesRegressor(n_estimators=100, max_depth=5, random_state=42))])),
    
    # Бустинги
    ('XGBoost', XGBRegressor(n_estimators=100, max_depth=3, enable_categorical=True, random_state=42)),
    ('LGBM', LGBMRegressor(n_estimators=100, max_depth=3, random_state=42)),
        
    # KNN
    ('KNN', Pipeline([('preprocessor', preprocessor), ('model', KNeighborsRegressor())])),
]

# Генерируем OOF-предсказания для всех моделей
oof_predictions = {}
for name, model in models:
    try:
        oof_pred = cross_val_predict(model, X, y, cv=kf, n_jobs=-1, verbose=0)
        # Универсальный расчет RMSE для разных версий sklearn
        rmse = np.sqrt(mean_squared_error(y, oof_pred))
        oof_predictions[name] = oof_pred
        print(f"{name} | RMSE: {rmse:.2f}")
    except Exception as e:
        print(f"Ошибка в {name}: {str(e)}")
        continue

# Собираем все предсказания в одну матрицу
X_meta = np.column_stack(list(oof_predictions.values()))

Ridge | RMSE: 2077.70
Lasso | RMSE: 2078.64
ElasticNet | RMSE: 2202.70
RandomForest | RMSE: 2206.47
ExtraTrees | RMSE: 2199.43
XGBoost | RMSE: 1944.72
LGBM | RMSE: 1957.08
KNN | RMSE: 2678.91


In [56]:
from sklearn.model_selection import GridSearchCV

# Масштабируем мета-признаки (KNN чувствителен к масштабу)
scaler = StandardScaler()
X_meta_scaled = scaler.fit_transform(X_meta)

# Подбираем оптимальное число соседей
param_grid = {
    'n_neighbors': [3, 5, 7, 10, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn = KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid_search.fit(X_meta_scaled, y)

print("Лучшие параметры KNN:", grid_search.best_params_)
print("Лучший RMSE:", -grid_search.best_score_)

# Лучшая KNN-модель
best_knn = grid_search.best_estimator_

Лучшие параметры KNN: {'metric': 'euclidean', 'n_neighbors': 15, 'weights': 'distance'}
Лучший RMSE: 1959.814919793377


In [57]:
from sklearn.base import clone
from joblib import parallel_backend

# 1. Переобучаем все базовые модели на полных данных
final_models = {}
for name, model in models:
    try:
        print(f"\nПереобучение {name} на всех данных...")
        
        # Создаем новую копию модели
        if isinstance(model, Pipeline):
            # Для пайплайнов используем clone
            new_model = clone(model)
        else:
            # Для обычных моделей создаем новый экземпляр с теми же параметрами
            new_model = model.__class__(**model.get_params())
        
        # Обучение с прогресс-баром для XGBoost/LGBM/CatBoost
        if name in ['XGBoost', 'LGBM']:
            with parallel_backend('threading', n_jobs=-1):
                new_model.fit(X, y)
        else:
            new_model.fit(X, y)
            
        final_models[name] = new_model
        print(f"{name} успешно переобучен!")
        
    except Exception as e:
        print(f"Ошибка при переобучении {name}: {str(e)}")
        continue

# 2. Функция для предсказаний на новых данных
def stack_predict(X_new):
    """
    Предсказание с использованием стекинга
    
    Параметры:
    X_new - новые данные (DataFrame или array-like)
    
    Возвращает:
    Предсказания мета-модели
    """
    # Собираем предсказания всех моделей
    preds = []
    for name, model in final_models.items():
        try:
            if name == 'XGBoost':
                # Особый случай для XGBoost с категориями
                X_new_xgb = X_new.copy()
                for col in categorical_cols:
                    if col in X_new_xgb.columns:
                        X_new_xgb[col] = X_new_xgb[col].astype('category')
                pred = model.predict(X_new_xgb)
            else:
                pred = model.predict(X_new)
            preds.append(pred)
        except Exception as e:
            print(f"Ошибка в {name} при предсказании: {str(e)}")
            continue
    
    # Преобразуем в матрицу и масштабируем
    X_meta_new = np.column_stack(preds)
    X_meta_new_scaled = scaler.transform(X_meta_new)
    
    # Делаем предсказание
    return best_knn.predict(X_meta_new_scaled)

# 3. Пример использования на тестовых данных
if 'test' in locals():
    try:
        print("\nПрогнозирование на тестовых данных...")
        test_predictions = stack_predict(test)
        print("Предсказания готовы! Пример первых 5 значений:")
        print(test_predictions[:5])
        
        # Сохранение результатов
        pd.DataFrame({'prediction': test_predictions}).to_csv('stacking_predictions.csv', index=False)
        print("Предсказания сохранены в stacking_predictions.csv")
    except Exception as e:
        print(f"Ошибка при прогнозировании на тесте: {str(e)}")


Переобучение Ridge на всех данных...
Ridge успешно переобучен!

Переобучение Lasso на всех данных...
Lasso успешно переобучен!

Переобучение ElasticNet на всех данных...
ElasticNet успешно переобучен!

Переобучение RandomForest на всех данных...
RandomForest успешно переобучен!

Переобучение ExtraTrees на всех данных...
ExtraTrees успешно переобучен!

Переобучение XGBoost на всех данных...
XGBoost успешно переобучен!

Переобучение LGBM на всех данных...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014415 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3648
[LightGBM] [Info] Number of data points in the train set: 120094, number of used features: 129
[LightGBM] [Info] Start training from score 3042.230809
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

In [58]:
ans = pd.DataFrame({})
ans['index'] = index
ans['target'] = test_predictions

In [60]:
ans.to_csv('ansamble.csv', index=False)

In [64]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# 1. Загрузка данных
train = pd.read_csv('train_contest.csv')
test = pd.read_csv('test_contest.csv')

# 2. Разделение
y = train['target']
X = train.drop(columns=['target'])
test_index = test['index']
X_test = test.drop(columns=['index'])


# 3. Категориальные признаки
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_cols:
    X[col] = X[col].astype('category')
    X_test[col] = X_test[col].astype('category')

# 4. Базовые модели
base_models = {
    'LGBM': LGBMRegressor(n_estimators=300, max_depth=6, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=300, max_depth=6, enable_categorical=True, random_state=42),
    'CatBoost': CatBoostRegressor(n_estimators=300, depth=6, verbose=0, cat_features=categorical_cols, random_state=42)
}

# 5. Кросс-валидация для OOF-предсказаний
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = {}
test_preds = {}

for name, model in base_models.items():
    print(f"Тренировка модели: {name}")
    model_clone = clone(model)
    
    # Cross-val predict (OOF)
    oof_pred = cross_val_predict(model_clone, X, y, cv=kf, method='predict', n_jobs=-1)
    oof_preds[name] = oof_pred
    
    # Обучение на всем тренировочном наборе и предсказание на тесте
    model_clone.fit(X, y)
    test_preds[name] = model_clone.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y, oof_pred))
    print(f"{name} RMSE: {rmse:.2f}")

# 6. Формируем матрицу признаков для мета-модели
X_meta = np.column_stack([oof_preds[m] for m in base_models])
X_test_meta = np.column_stack([test_preds[m] for m in base_models])

# Масштабирование метапризнаков
scaler = StandardScaler()
X_meta_scaled = scaler.fit_transform(X_meta)
X_test_meta_scaled = scaler.transform(X_test_meta)

# 7. Мета-модель (Ridge)
meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta_scaled, y)
final_preds = meta_model.predict(X_test_meta_scaled)

meta_rmse = np.sqrt(mean_squared_error(y, cross_val_predict(meta_model, X_meta_scaled, y, cv=kf)))
print(f"Метамодель Ridge RMSE (по OOF): {meta_rmse:.2f}")

# 8. Сохранение результата
submission = pd.DataFrame({
    'index': test_index,
    'target': final_preds
})
submission.to_csv('stacking_submission.csv', index=False)
print("Файл 'stacking_submission.csv' успешно сохранен.")


Тренировка модели: LGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014032 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3648
[LightGBM] [Info] Number of data points in the train set: 120094, number of used features: 129
[LightGBM] [Info] Start training from score 3042.230809
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w